In [5]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import os

In [6]:
def save_world_to_database(df_world):
    connection_string = "postgres:postgres@localhost:5432/corona_db"
    engine = create_engine(f'postgresql://{connection_string}')
    connection = engine.connect()
    connection.execute( '''TRUNCATE TABLE daily_stats_world''' )
    connection.close()
    df_world.to_sql(name='daily_stats_world', con=engine, if_exists='append', index=False)

In [7]:
# Start here to process the world data (only call save_world_to_database)
url_world = "static/world_data/df_world_all.csv"
df_world = pd.read_csv(url_world)
df_world = df_world.drop(['country_2'], axis=1)
df_world.head()
save_world_to_database(df_world)

In [8]:
# query_str = open('static/sql/test_world.sql')
query_str = open('static/sql/world_top_10.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')    
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query


select  
	country, 
	sum(conf_count) total,
	sum(conf_count)-sum(cured_count)-sum(dead_count) sick, 
	sum(cured_count) cured, 
	sum(dead_count) dead,
	max(date)
    from daily_stats_world 
    where date = (select max(date ) from daily_stats_world) 
    group by country 
    order by total desc 
	limit 10


,country,total,sick,cured,dead,max
0,China,80151,29802,47404,2945,2020-03-03
1,South Korea,5186,5128,30,28,2020-03-03
2,Italy,2502,2263,160,79,2020-03-03
3,Iran,2336,1968,291,77,2020-03-03
4,Others,706,690,10,6,2020-03-03
5,Japan,293,244,43,6,2020-03-03
6,France,204,188,12,4,2020-03-03
7,Germany,196,180,16,0,2020-03-03
8,Spain,165,162,2,1,2020-03-03
9,United States of America,122,107,8,7,2020-03-03
